In [1]:
import sys
import numpy as np
import pandas as pd
import sqlite3 as sql
import ast

# from sklearn import linear_model

strDir = './output/'
strCSVFile = '20170322_pmid_summaries_concussion.csv'
# strCSVFile = '20170322_traumatic_brain_injury_pmid_summaries.csv'
# Set the name of the CSV file to arg if possible
# if len(sys.argv) > 1:
#     strCSVFile = sys.argv[1]

df = pd.read_csv(strDir + strCSVFile, parse_dates=[4,18,23]) # 4, 18, 23 are dates

# df = pd.DataFrame(df[0:10], copy=True)

# df['PubDate'] = pd.to_datetime(df['EPubDate'], errors='coerce')
# df['PubDate'] = pd.to_datetime(df['PubDate'], errors='coerce')
# df['SO'] = pd.to_datetime(df['SO'], errors='coerce')
# print(df.head)
# print(df.PubDate.dt.day)

strSQLFile = './output/citations.sqlite'
conn = sql.connect(strSQLFile)
c = conn.cursor()

# Add Papers table
strSQL = 'DROP TABLE IF EXISTS "tblPapers"'
c.execute(strSQL) 
strSQL = 'CREATE TABLE "tblPapers" (\
            "Paper_ID" INTEGER PRIMARY KEY AUTOINCREMENT, \
            "ArticleIds" TEXT, "DOI" TEXT, "ELocationID" TEXT, "EPubDate" TEXT, "ESSN" TEXT, \
            "FullJournalName" TEXT, "HasAbstract" INTEGER, "History" TEXT, \
            "ISSN" TEXT, "Id" INTEGER, "Issue" TEXT, "Item" TEXT, \
            "LastAuthor" TEXT, "NlmUniqueID" TEXT, "Pages" TEXT, \
            "PmcRefCount" TEXT, "PubDate" TEXT, "Title" TEXT, "Volume" INTEGER, "NumOfAuthors" INTEGER \
            )'
c.execute(strSQL)

# Add Authors table
strSQL = 'DROP TABLE IF EXISTS "tblAuthors"'
c.execute(strSQL) 
strSQL = 'CREATE TABLE "tblAuthors" (\
            "Author_ID" INTEGER PRIMARY KEY AUTOINCREMENT, \
            "Paper_ID" INTEGER, "Author" TEXT \
            )'
c.execute(strSQL)

# Add Languages table
strSQL = 'DROP TABLE IF EXISTS "tblLanguages"'
c.execute(strSQL) 
strSQL = 'CREATE TABLE "tblLanguages" (\
            "Language_ID" INTEGER PRIMARY KEY AUTOINCREMENT, \
            "Paper_ID" INTEGER, "Language" TEXT \
            )'
c.execute(strSQL)

# Add Pub Type table
strSQL = 'DROP TABLE IF EXISTS "tblPubType"'
c.execute(strSQL) 
strSQL = 'CREATE TABLE "tblPubType" (\
            "PubType_ID" INTEGER PRIMARY KEY AUTOINCREMENT, \
            "Paper_ID" INTEGER, "PubType" TEXT \
            )'
c.execute(strSQL)

for row in df.itertuples(index=False, name='Pandas'):
    
#     print(row.ArticleIds)
#     vIDs = ast.literal_eval(row.ArticleIds)

    # Get list of authors to add count to tblPapers 
    vAuthors = ast.literal_eval(row.AuthorList)
    
    strTitle = row.Title
    if isinstance(strTitle,float):
        strTitle = "[BLANK]"
    strTitle = strTitle.replace("\"","")
        
    # Add each paper to the Papers table
    strSQL = ( "INSERT INTO \"tblPapers\" \
            (\"ArticleIds\", \"DOI\", \"ELocationID\", \"EPubDate\", \"ESSN\", \
            \"FullJournalName\", \"HasAbstract\", \"History\", \
            \"ISSN\", \"Id\", \"Issue\", \"Item\", \
            \"LastAuthor\", \"NlmUniqueID\", \"Pages\", \
            \"PmcRefCount\", \"PubDate\", \"Title\", \"Volume\", \"NumOfAuthors\") \
            VALUES (\"%s\", \"%s\", \"%s\", \"%s\", \"%s\", \"%s\", \"%d\", \"%s\", \"%s\", \
                \"%s\", \"%s\", \"%s\", \"%s\", \"%s\", \"%s\", \"%d\", \"%s\", \"%s\", \"%s\", %d);" \
              % (row.ArticleIds, row.DOI, row.ELocationID, row.EPubDate, row.ESSN, \
                 row.FullJournalName, row.HasAbstract, row.History, \
                 row.ISSN, row.Id, row.Issue, row.Item, \
                 row.LastAuthor, row.NlmUniqueID, row.Pages, \
                 row.PmcRefCount, row.PubDate, \
                 strTitle, row.Volume, len(vAuthors) ))
    
#     print(strSQL)
    c.execute(strSQL)
    iPaperID = c.lastrowid # Get the id of the newly added row
    
    # For each author in the paper, add the author into the Authors table
    for strAuthor in vAuthors:
        strSQL = ( "INSERT INTO \"tblAuthors\" \
            (\"Paper_ID\", \"Author\") \
            VALUES (%d, \"%s\");" % (iPaperID, strAuthor) )
        c.execute(strSQL)
    
    # For each language
    vLangs = ast.literal_eval(row.LangList)
    for strLang in vLangs:
        strSQL = ( "INSERT INTO \"tblLanguages\" \
            (\"Paper_ID\", \"Language\") \
            VALUES (%d, \"%s\");" % (iPaperID, strLang) )
        c.execute(strSQL)
        
    # For each publication type
    vPubTypes = ast.literal_eval(row.PubTypeList)
    for strPubType in vPubTypes:
        strSQL = ( "INSERT INTO \"tblPubType\" \
            (\"Paper_ID\", \"PubType\") \
            VALUES (%d, \"%s\");" % (iPaperID, strPubType) )
        c.execute(strSQL)
    
conn.commit()
conn.close()

